In [5]:
import os
import shutil
import sys
sys.path.append('/storage/home/lam7027/work/bin/vasp-job-automation')
import prep_for_vasp as pforv
from pymatgen.core.structure import Structure



In [2]:
potcar = 'POTCAR'
kpoints = 'KPOINTS'
pforv.parse('YWoutput')
atom_count_df = pforv.count_atoms()
pforv.remove_spin_up_less_than_down(atom_count_df, 'Fe+', 'Fe-')
pforv.convert_strs_to_poscars()
pforv.make_incars('Fe+', 'Fe-', 'Se', '5', '-5', '0', 'INCAR')
for dir in os.listdir('configurations'):
    shutil.copy(potcar, os.path.join('configurations', dir))


In [3]:
config_dirs = [os.path.join('configurations', dir) for dir in os.listdir('configurations')]
for config_dir in config_dirs:
    pforv.rearrage_sites_and_magmoms(config_dir)

In [2]:
pforv.scale_poscars(21)

In [7]:
struct = Structure.from_file('/storage/home/lam7027/work/FeSe/genstrs_all/configurations/config_1031/POSCAR')
print(struct.volume)

336.0


In [ ]:
pforv.make_kpoints(6000)

In [ ]:
pforv.create_submit_scripts(configurations_directory='configurations', submit_script='run_vasp',)